In [3]:
import boto3
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv

In [15]:
bucket_name = "kayak-final-projet"  # Remplace avec ton bucket S3
file_name = "geolocalisation.csv"  # Le fichier local
s3_key = file_name # Chemin dans S3 (optionnel)

In [12]:
# Charger les variables du fichier .env
load_dotenv()

# Récupérer les valeurs des variables
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
aws_bd_password = os.getenv("DB_PASSWORD")

# Initialiser le client Boto3 avec les credentials
s3_client = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=aws_region,
)

In [7]:

# Remplace ces valeurs par les tiennes
DB_HOST = "kayak.clwqcisew0ya.eu-west-3.rds.amazonaws.com"
DB_PORT = 5432  # 3306 pour MySQL, 5432 pour PostgreSQL
DB_NAME = "postgres"
DB_USER = "postgres"

def test_connection():
    try:
        conn = psycopg2.connect(
                host=DB_HOST,
                port=DB_PORT,
                user=DB_USER,
                password="Wailou2825!",
                dbname=DB_NAME
            )        
        print("Connexion réussie !")
        conn.close()
    
    except Exception as e:
        print(f"Erreur de connexion : {e}")

if __name__ == "__main__":
    test_connection()


Connexion réussie !


In [8]:
def create_table():
    try:
        conn = psycopg2.connect(
                host=DB_HOST,
                port=DB_PORT,
                user=DB_USER,
                password="Wailou2825!",
                dbname=DB_NAME
            ) 
        cur = conn.cursor()

        cur.execute("""
            CREATE TABLE IF NOT EXISTS villes (
                id SERIAL PRIMARY KEY,
                villes TEXT NOT NULL,
                longitude FLOAT NOT NULL,
                latitude FLOAT NOT NULL
            )
        """)

        conn.commit()
        cur.close()
        conn.close()
        print("Table créée avec succès !")

    except Exception as e:
        print(f"Erreur lors de la création de la table : {e}")

create_table()

Table créée avec succès !


In [9]:
def lambda_handler():
    
    # Télécharger le fichier depuis S3
    # obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv("geolocalisation.csv", sep=";")

    # Connexion à la base RDS
    conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            user=DB_USER,
            password="Wailou2825!",
            dbname=DB_NAME
        ) 

    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            cursor.execute("INSERT INTO villes (villes, longitude, latitude) VALUES (%s, %s, %s)", (row['adresse'], float(row['longitude']), float(row['latitude'])))
        
    conn.commit()
    conn.close()


In [10]:
lambda_handler()